## Training on TPU

### Install packages

In [1]:
%%capture
!pip install -U efficientnet

### Imports

In [2]:
from pathlib import Path
from functools import partial

import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import wandb
from wandb.keras import WandbCallback
from sklearn import model_selection

import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets

## WandB Login

In [3]:
!wandb login f137298421da563b24639d1287dd3ce5da537814

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [4]:
wandb.init(project="kaggle-melanoma")

wandb: Wandb version 0.8.36 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


W&B Run: https://app.wandb.ai/nisarahamedk/kaggle-melanoma/runs/28vhzrb5

## TPU Config

Detect hardware and return appropriate distribution strategy

In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # no parameter needed for TPU_NAME env variable is set. This is the case for Kaggle
    print("Running on TPU: ", tpu.master())
except ValueError:
    tpu = None

Running on TPU:  grpc://10.0.0.2:8470


In [6]:
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # default strategy with the available hw
    strategy = tf.distribute.get_strategy()
    
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  8


## Dataset from GCS for TPU

In [7]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path("siim-isic-melanoma-classification")

In [8]:
GCS_DS_PATH

'gs://kds-f941eda182a90288c86bd91ec807ec5f6c58e5c4e0fbf9ad2114f762'

In [ ]:
!gsutil ls $GCS_DS_PATH # list the GCS bucket

In [9]:
train_files = tf.io.gfile.glob(GCS_DS_PATH + "/tfrecords/train*")
test_files = tf.io.gfile.glob(GCS_DS_PATH + "/tfrecords/test*")

### Train, Valid split

In [10]:
LOCAL_DS_PATH = Path("/kaggle/input/siim-isic-melanoma-classification")

In [11]:
train_df = pd.read_csv(LOCAL_DS_PATH / "train.csv")
train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


Assign a fold id for each images using StratifiedKFold

In [12]:
train_df["kfold"] = -1

y = train_df["target"].values

skf = model_selection.StratifiedKFold(n_splits=5, shuffle=True)

for fold, (train_idx, test_idx) in enumerate(skf.split(X=train_df, y=y)):
    train_df.loc[test_idx, "kfold"] = fold
    
train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,kfold
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,4
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,4
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,1
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,3
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,0


This way, when we run training with fold=1, images with column "kfold=1" will be kept in validation set, others in training set.

In [13]:
train_df["kfold"].value_counts()

0    6626
4    6625
3    6625
2    6625
1    6625
Name: kfold, dtype: int64

### tf.Dataset pipeline 

Building the complete tfrecord -> model data pipeline

In [ ]:
train_dataset = tf.data.TFRecordDataset(train_files, num_parallel_reads=tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.TFRecordDataset(test_files, num_parallel_reads=tf.data.experimental.AUTOTUNE)

* Checking the feature discription of the tfreceord files

We have features: "image", "image_name" and "target"

In [ ]:
# test set
for item in test_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(item.numpy())
    print(example)

We have "image" and "image_name" for the test dataset

#### Feature description


In [14]:
train_feature_desc = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "image_name": tf.io.FixedLenFeature([], tf.string), # for filtering images belong to val set.
    "target": tf.io.FixedLenFeature([], tf.int64),
}

test_feature_desc = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "image_name": tf.io.FixedLenFeature([], tf.string),
}

Using the above feature description, 
* Lets load the dataset from the tfrecord files
* Process it using the feature description.
* Decode each sample into an image.
* return image, target pairs  

*Read from bottom to top*

In [15]:
TPU_IMAGE_SIZE = 1024
INPUT_IMAGE_SIZE = 512

In [16]:
fold = train_df[["image_name", "kfold"]].set_index("image_name").to_dict()
fold = fold["kfold"]
fold

{'ISIC_2637011': 4,
 'ISIC_0015719': 4,
 'ISIC_0052212': 1,
 'ISIC_0068279': 3,
 'ISIC_0074268': 0,
 'ISIC_0074311': 4,
 'ISIC_0074542': 1,
 'ISIC_0075663': 0,
 'ISIC_0075914': 4,
 'ISIC_0076262': 0,
 'ISIC_0076545': 2,
 'ISIC_0076742': 3,
 'ISIC_0076995': 4,
 'ISIC_0077472': 0,
 'ISIC_0077735': 2,
 'ISIC_0078703': 1,
 'ISIC_0078712': 1,
 'ISIC_0079038': 0,
 'ISIC_0080512': 1,
 'ISIC_0080752': 3,
 'ISIC_0080817': 0,
 'ISIC_0081956': 0,
 'ISIC_0082348': 0,
 'ISIC_0082543': 3,
 'ISIC_0082934': 1,
 'ISIC_0083035': 3,
 'ISIC_0084086': 1,
 'ISIC_0084270': 4,
 'ISIC_0084395': 0,
 'ISIC_0085172': 1,
 'ISIC_0085718': 1,
 'ISIC_0085902': 0,
 'ISIC_0086349': 4,
 'ISIC_0086462': 4,
 'ISIC_0086632': 3,
 'ISIC_0086709': 2,
 'ISIC_0087290': 3,
 'ISIC_0087297': 0,
 'ISIC_0088137': 4,
 'ISIC_0088489': 3,
 'ISIC_0089401': 0,
 'ISIC_0089569': 3,
 'ISIC_0089738': 2,
 'ISIC_0090279': 0,
 'ISIC_0091016': 0,
 'ISIC_0091559': 2,
 'ISIC_0091996': 2,
 'ISIC_0092726': 3,
 'ISIC_0092870': 4,
 'ISIC_0094701': 3,


In [17]:
def decode_image(image_data):
    image_size = [TPU_IMAGE_SIZE, TPU_IMAGE_SIZE]
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*image_size, 3])  # explicit size needed for TPU
    image = tf.image.resize(image, [INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE])
    return image
                           
def parse_test_example(example):
    """ function to parse each example read from the test tfrecord file"""
    example = tf.io.parse_single_example(example, test_feature_desc)
    return example

def parse_train_example(example):
    """ function to parse each example read from the train tfrecord file"""
    example = tf.io.parse_single_example(example, train_feature_desc)
    return example

def process_train_example(example):
    image = decode_image(example["image"])
    label = tf.cast(example["target"], tf.int32)
    return image, label

def process_test_example(example):
    image = decode_image(example["image"])
    image_name = example["image_name"]
    return image, image_name

def train_filter_fn(example):
    # convert folds dict to tensorflow lookup table.
    img_names = tf.constant(list(fold.keys()))
    fold_idx = tf.constant(list(fold.values()))
    folds_init = tf.lookup.KeyValueTensorInitializer(img_names, fold_idx)
    folds_table = tf.lookup.StaticHashTable(folds_init, -1)
    return folds_table.lookup(example["image_name"]) != 1
    
def valid_filter_fn(example):
    # convert folds dict to tensorflow lookup table.
    img_names = tf.constant(list(fold.keys()))
    fold_idx = tf.constant(list(fold.values()))
    folds_init = tf.lookup.KeyValueTensorInitializer(img_names, fold_idx)
    folds_table = tf.lookup.StaticHashTable(folds_init, -1)
    return folds_table.lookup(example["image_name"]) == 1
    
def load_dataset_from_tfrecord(filenames, ds_type="train", ordered=False):
    
    # Since we are reading dataset from multiple files. and we dont care about the order.
    # set deterministic reading to False.
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
    dataset.with_options(ignore_order)
    
    # parse each example with feature description
    if ds_type in ["train", "valid"]:
        dataset = dataset.map(parse_train_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        # filter  
        dataset = dataset.filter(train_filter_fn if ds_type=="train" else valid_filter_fn)
        dataset = dataset.map(process_train_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    else:
        dataset = dataset.map(parse_test_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.map(process_test_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    return dataset
    

Data augmentation for the image

In [18]:
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

Finally, the datapipeline function that puts these all together

In [19]:
batch_size = 16 * strategy.num_replicas_in_sync
def get_training_dataset():
    dataset = load_dataset_from_tfrecord(train_files, ds_type="train")
    dataset = dataset.map(data_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [20]:
def get_validation_dataset():
    dataset = load_dataset_from_tfrecord(train_files, ds_type="valid")
    dataset = dataset.map(data_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [21]:
def get_test_dataset(ordered=False):
    dataset = load_dataset_from_tfrecord(test_files, ds_type="test", ordered=ordered)
    dataset = dataset.batch(batch_size)
    dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

Sanity check

In [22]:
print("*** Training set shapes *****")
for image, label in get_training_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
    
print("*** Training set labels: ", label.numpy())

print("*** Validation set shapes *****")
for image, label in get_validation_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
    
print("*** Validation set labels: ", label.numpy())


print("*** Test set shape ***")
for image, image_name in get_test_dataset().take(3):
    print(image.numpy().shape, image_name.numpy().shape)
print("*** Test set image_name: ", image_name.numpy().astype("U"))

*** Training set shapes *****


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:60: UserWarning: Creating resources inside a function passed to Dataset.filter() is not supported. Create each resource outside the function, and capture it inside the function to use it.


(128, 512, 512, 3) (128,)
(128, 512, 512, 3) (128,)
(128, 512, 512, 3) (128,)
*** Training set labels:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
*** Validation set shapes *****
(128, 512, 512, 3) (128,)
(128, 512, 512, 3) (128,)
(128, 512, 512, 3) (128,)
*** Validation set labels:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
*** Test set shape ***
(128, 512, 512, 3) (128,)
(128, 512, 512, 3) (128,)
(128, 512, 512, 3) (128,)
*** Test set image_name:  ['ISIC_3009035' 'ISIC_1579773' 'ISIC_6082685' 'ISIC_1263999'
 'ISIC_4348477' 'ISIC_3740111' 'ISIC_1071664' 'I

## Model - Efficient Net 

In [ ]:
with strategy.scope():
    model = keras.Sequential([
        efn.EfficientNetB7(
            input_shape=[INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE, 3],
            weights="imagenet",
            include_top=False,
        ),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(1024, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(1, activation="sigmoid"),
    ])

258441216/258434480 [==============================] - 3s 0us/step


In [ ]:
model.compile(
    optimizer="adam", 
    # loss="binary_crossentropy", 
    loss=tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.1), 
    metrics=["accuracy", keras.metrics.AUC()]
)

model.summary()

## Training

In [ ]:
history = model.fit(get_training_dataset(), validation_data=get_validation_dataset(), epochs=5, callbacks=[WandbCallback()])

## Submission

In [ ]:
test_ds = get_test_dataset(ordered=True)

print("Computing predictions...")
test_image_ds = test_ds.map(lambda image, image_name: image)
probs = model.predict(test_image_ds).flatten()
print(probs)

In [ ]:
import re
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
print("Generating submission file")
num_test_images = count_data_items(test_files)
test_image_names_ds = test_ds.map(lambda image, image_name: image_name).unbatch()

test_image_names = next(iter(test_image_names_ds.batch(num_test_images))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_image_names, probs]), fmt=['%s', '%f'], delimiter=',', header='image_name,target', comments='')
!head submission.csv